Trying to see if my Anyscale docs are going to work with the existing code because I downloaded them as HTMLs (and curated on top of that). Note to self: we have to find a way to systemetize the uploading of HTMLs and I suggest doing a git clone, npm run build, and then taking the build output minus some custom pages to index.

In [ ]:
def extract_text_from_section(section):
    texts = []
    for elem in section.children:
        if isinstance(elem, NavigableString):
            if elem.strip():
                texts.append(elem.strip())
        elif elem.name == "section":
            continue
        else:
            texts.append(elem.get_text().strip())
    return "\n".join(texts)


def path_to_uri(path, scheme="https://", domain="docs.ray.io"):
    return scheme + domain + str(path).split(domain)[-1]


def extract_sections(record):
    with open(record["path"], "r", encoding="utf-8") as html_file:
        soup = BeautifulSoup(html_file, "html.parser")
    sections = soup.find_all("section")
    section_list = []
    for section in sections:
        section_id = section.get("id")
        section_text = extract_text_from_section(section)
        if section_id:
            uri = path_to_uri(path=record["path"])
            section_list.append({"source": f"{uri}#{section_id}", "text": section_text})
    return section_list

In [ ]:
# Imports
import os
import ray
import sys; sys.path.append("..")
import warnings; warnings.filterwarnings("ignore")
from dotenv import load_dotenv; load_dotenv()
%load_ext autoreload
%autoreload 2
from rag.config import ROOT_DIR

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Start the Ray cluster, with relevant credentials; we're not using Anyscale Endpoints.

ray.init(runtime_env={
    "env_vars": {
        "OPENAI_API_BASE": os.environ["OPENAI_API_BASE"],
        "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"], 
        "DB_CONNECTION_STRING": os.environ["DB_CONNECTION_STRING"],
    },
    "working_dir": str(ROOT_DIR)
})

2024-08-30 10:49:18,397	INFO worker.py:1598 -- Connecting to existing Ray cluster at address: 10.0.13.134:6379...
2024-08-30 10:49:18,406	INFO worker.py:1774 -- Connected to Ray cluster. View the dashboard at https://session-vig1su7dbnvzqbs3l1tw35fhk3.i.anyscaleuserdata-staging.com 
2024-08-30 10:49:18,571	INFO packaging.py:530 -- Creating a file package for local directory '/home/ray/default/anyscale-ragbot/notebooks/..'.
2024-08-30 10:49:18,814	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_5d0a64a022ae6e67.zip' (47.01MiB) to Ray cluster...
2024-08-30 10:49:19,363	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_5d0a64a022ae6e67.zip'.


Python version:,3.12.4
Ray version:,2.35.0
Dashboard:,http://session-vig1su7dbnvzqbs3l1tw35fhk3.i.anyscaleuserdata-staging.com


(autoscaler +2m22s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.


In [ ]:
# Only text-embedding-3-large dimension and gpt-4o context length relevant for now.
from rag.config import EMBEDDING_DIMENSIONS, MAX_CONTEXT_LENGTHS

In [ ]:
from pathlib import Path
from rag.config import EFS_DIR

ANYSCALE_DOCS_DIR = Path(EFS_DIR, "docs.anyscale.com/")
ANYSCALE_DOCS_URL = "https://docs.anyscale.com"

In [ ]:
ANYSCALE_DOCS_DIR

PosixPath('/mnt/shared_storage/emmy/docs.anyscale.com')

In [ ]:
ds = ray.data.from_items(
        [{"path": path} for path in ANYSCALE_DOCS_DIR.rglob("*.html") if not path.is_dir()]
    )

In [ ]:
print(f"{ds.count()} documents")

244 documents


In [ ]:
ds.show(5)

2024-08-30 12:42:06,883	INFO dataset.py:2409 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
2024-08-30 12:42:06,887	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-08-30_10-13-10_110991_2828/logs/ray-data
2024-08-30 12:42:06,888	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> LimitOperator[limit=5]


Running 0: 0.00 row [00:00, ? row/s]

- limit=5 1: 0.00 row [00:00, ? row/s]

{'path': PosixPath('/mnt/shared_storage/emmy/docs.anyscale.com/migration_v2.html')}
{'path': PosixPath('/mnt/shared_storage/emmy/docs.anyscale.com/index.html')}
{'path': PosixPath('/mnt/shared_storage/emmy/docs.anyscale.com/get-started.html')}
{'path': PosixPath('/mnt/shared_storage/emmy/docs.anyscale.com/ci-cd/github-actions.html')}
{'path': PosixPath('/mnt/shared_storage/emmy/docs.anyscale.com/ci-cd/apache-airflow.html')}


In [ ]:
from bs4 import BeautifulSoup, NavigableString

In [ ]:
sample_html_fp = Path(EFS_DIR, "docs.ray.io/en/master/rllib/rllib-env.html")
extract_sections({"path": sample_html_fp})[0]

{'source': 'https://docs.ray.io/en/master/rllib/rllib-env.html#environments',
 'text': '\nEnvironments#\nRLlib works with several different types of environments, including Farama-Foundation Gymnasium, user-defined, multi-agent, and also batched environments.\nTip\nNot all environments work with all algorithms. See the algorithm overview for more information.\n'}

In [ ]:
test_path = Path(EFS_DIR, "docs.anyscale.com/ci-cd/apache-airflow/index.html")
extract_sections({"path": test_path})[0]

IndexError: list index out of range

In [ ]:
sections_ds = ds.flat_map(extract_sections)

In [ ]:

    sections_ds = ds.flat_map(extract_sections)
    chunks_ds = sections_ds.flat_map(
        partial(chunk_section, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    )

In [ ]:
def chunk_section(section, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", " ", ""],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    chunks = text_splitter.create_documents(
        texts=[section["text"]], metadatas=[{"source": section["source"]}]
    )
    return [{"text": chunk.page_content, "source": chunk.metadata["source"]} for chunk in chunks]
